# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [6]:
y_val = housing['medianHouseValue']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf

In [16]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [17]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [19]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\TINYAN~1\\AppData\\Local\\Temp\\tmprjl5855m', '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_session_config': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_save_checkpoints_secs': 600}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [20]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\TINYAN~1\AppData\Local\Temp\tmprjl5855m\model.ckpt.
INFO:tensorflow:loss = 5.31703e+11, step = 1
INFO:tensorflow:global_step/sec: 475.969
INFO:tensorflow:loss = 3.95881e+11, step = 101 (0.202 sec)
INFO:tensorflow:global_step/sec: 378.505
INFO:tensorflow:loss = 2.93873e+11, step = 201 (0.265 sec)
INFO:tensorflow:global_step/sec: 442.49
INFO:tensorflow:loss = 5.01389e+11, step = 301 (0.225 sec)
INFO:tensorflow:global_step/sec: 423.809
INFO:tensorflow:loss = 2.71964e+11, step = 401 (0.238 sec)
INFO:tensorflow:global_step/sec: 395.701
INFO:tensorflow:loss = 3.99608e+11, step = 501 (0.245 sec)
INFO:tensorflow:global_step/sec: 439.543
INFO:tensorflow:loss = 6.52303e+11, step = 601 (0.231 sec)
INFO:tensorflow:global_step/sec: 418.065
INFO:tensorflow:loss = 3.10814e+11, step = 701 (0.237 sec)
INFO:tensorflow:global_step/sec: 463.031
INFO:tensorflow:loss = 2.48224e+11, step = 801 (0.219 sec)
INFO:

INFO:tensorflow:loss = 5.41482e+10, step = 8001 (0.170 sec)
INFO:tensorflow:global_step/sec: 607.618
INFO:tensorflow:loss = 4.79382e+10, step = 8101 (0.162 sec)
INFO:tensorflow:global_step/sec: 615.31
INFO:tensorflow:loss = 9.70909e+10, step = 8201 (0.178 sec)
INFO:tensorflow:global_step/sec: 499.985
INFO:tensorflow:loss = 6.45027e+10, step = 8301 (0.184 sec)
INFO:tensorflow:global_step/sec: 593.672
INFO:tensorflow:loss = 9.80951e+10, step = 8401 (0.185 sec)
INFO:tensorflow:global_step/sec: 555.58
INFO:tensorflow:loss = 1.30601e+11, step = 8501 (0.171 sec)
INFO:tensorflow:global_step/sec: 499.02
INFO:tensorflow:loss = 1.46893e+11, step = 8601 (0.204 sec)
INFO:tensorflow:global_step/sec: 502.201
INFO:tensorflow:loss = 7.23083e+10, step = 8701 (0.197 sec)
INFO:tensorflow:global_step/sec: 472.321
INFO:tensorflow:loss = 9.42508e+10, step = 8801 (0.213 sec)
INFO:tensorflow:global_step/sec: 546.964
INFO:tensorflow:loss = 1.02528e+11, step = 8901 (0.173 sec)
INFO:tensorflow:global_step/sec: 5

INFO:tensorflow:loss = 1.17558e+11, step = 16101 (0.193 sec)
INFO:tensorflow:global_step/sec: 510.226
INFO:tensorflow:loss = 4.15956e+10, step = 16201 (0.179 sec)
INFO:tensorflow:global_step/sec: 535.229
INFO:tensorflow:loss = 1.16479e+11, step = 16301 (0.186 sec)
INFO:tensorflow:global_step/sec: 606.595
INFO:tensorflow:loss = 6.4953e+10, step = 16401 (0.161 sec)
INFO:tensorflow:global_step/sec: 539.239
INFO:tensorflow:loss = 4.57302e+10, step = 16501 (0.201 sec)
INFO:tensorflow:global_step/sec: 526.708
INFO:tensorflow:loss = 8.93906e+10, step = 16601 (0.180 sec)
INFO:tensorflow:global_step/sec: 562.723
INFO:tensorflow:loss = 7.14197e+10, step = 16701 (0.172 sec)
INFO:tensorflow:global_step/sec: 556.684
INFO:tensorflow:loss = 1.42365e+11, step = 16801 (0.180 sec)
INFO:tensorflow:global_step/sec: 539.884
INFO:tensorflow:loss = 1.07903e+11, step = 16901 (0.185 sec)
INFO:tensorflow:global_step/sec: 526.919
INFO:tensorflow:loss = 1.0743e+11, step = 17001 (0.210 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 566.095
INFO:tensorflow:loss = 2.33529e+11, step = 24201 (0.167 sec)
INFO:tensorflow:global_step/sec: 566.668
INFO:tensorflow:loss = 5.31645e+10, step = 24301 (0.180 sec)
INFO:tensorflow:global_step/sec: 566.779
INFO:tensorflow:loss = 1.30452e+11, step = 24401 (0.172 sec)
INFO:tensorflow:global_step/sec: 432.109
INFO:tensorflow:loss = 4.59742e+10, step = 24501 (0.231 sec)
INFO:tensorflow:global_step/sec: 592.918
INFO:tensorflow:loss = 5.117e+10, step = 24601 (0.185 sec)
INFO:tensorflow:global_step/sec: 559.496
INFO:tensorflow:loss = 4.67806e+10, step = 24701 (0.163 sec)
INFO:tensorflow:global_step/sec: 453.727
INFO:tensorflow:loss = 2.65373e+11, step = 24801 (0.222 sec)
INFO:tensorflow:global_step/sec: 565.028
INFO:tensorflow:loss = 8.02709e+10, step = 24901 (0.175 sec)
INFO:tensorflow:Saving checkpoints for 25000 into C:\Users\TINYAN~1\AppData\Local\Temp\tmprjl5855m\model.ckpt.
INFO:tensorflow:Loss for final step: 9.07702e+10.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [21]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [22]:
pred_gen = model.predict(predict_input_func)

In [23]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\TINYAN~1\AppData\Local\Temp\tmprjl5855m\model.ckpt-25000


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [24]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_test,final_preds)**0.5

98666.17899100158

# Great Job!